在上一個模塊中，我們一直在使用文本的豐富語義表示，以及在嵌入之上的簡單線性分類器。這種架構的作用是捕獲句子中單詞的聚合含義，但它沒有考慮單詞的**順序**，因為嵌入之上的聚合操作從原始文本中刪除了這些信息。由於這些模型無法對詞序進行建模，因此它們無法解決更複雜或模棱兩可的任務，例如文本生成或問答。

為了捕捉文本序列的含義，我們需要使用另一種神經網絡架構，稱為**循環神經網絡**，或 RNN。在 RNN 中，我們一次一個符號地將我們的句子通過網絡，網絡產生一些**狀態**，然後我們用下一個符號再次將其傳遞給網絡。

![顯示循環神經網絡生成示例的圖像。](./img/sample-rnn-model-generation.png)

給定令牌 $X_0,\dots,X_n$ 的輸入序列，RNN 創建一個神經網絡塊序列，並使用反向傳播端到端地訓練這個序列。每個網絡塊以一對 $(X_i,S_i)$ 作為輸入，並產生 $S_{i+1}$ 作為結果。最終狀態 $S_n$ 或輸出 $X_n$ 進入線性分類器以產生結果。所有網絡塊共享相同的權重，並使用一次反向傳播進行端到端的訓練。

因為狀態向量 $S_0,\dots,S_n$ 是通過網絡傳遞的，所以它能夠學習單詞之間的順序依賴關係。例如，當單詞 *not* 出現在序列中的某處時，它可以學習否定狀態向量中的某些元素，從而產生否定。

讓我們看看循環神經網絡如何幫助我們對新聞數據集進行分類。

In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## 簡單的RNN分類器

在簡單 RNN 的情況下，每個循環單元都是一個簡單的線性網絡，它採用連接的輸入向量和狀態向量，並產生一個新的狀態向量。 PyTorch 用“RNNCell”類和此類單元的網絡表示該單元——作為“RNN”層。

為了定義一個 RNN 分類器，我們將首先應用一個嵌入層來降低輸入詞彙的維數，然後在它上面有一個 RNN 層：

In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **注意：** 為簡單起見，我們在這裡使用未經訓練的嵌入層，但為了獲得更好的結果，我們可以使用帶有 Word2Vec 或 GloVe 嵌入的預訓練嵌入層，如前一單元所述。為了更好地理解，您可能需要修改此代碼以使用預訓練的嵌入。

在我們的例子中，我們將使用填充數據加載器，因此每個批次都會有許多相同長度的填充序列。 RNN 層將採用嵌入張量的序列，並產生兩個輸出：
* $x$ 是每一步的 RNN 單元輸出序列
* $h$ 是序列最後一個元素的最終隱藏狀態

然後我們應用一個全連接的線性分類器來獲得類的數量。

> **注意：** RNN 很難訓練，因為一旦 RNN 單元沿著序列長度展開，反向傳播所涉及的層數就會非常大。因此我們需要選擇較小的學習率，並在較大的數據集上訓練網絡以產生良好的結果。這可能需要很長時間，因此首選使用 GPU。

In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3075
6400: acc=0.37328125
9600: acc=0.43604166666666666
12800: acc=0.482109375
16000: acc=0.5236875
19200: acc=0.5579166666666666
22400: acc=0.5868303571428571
25600: acc=0.6102734375
28800: acc=0.6298263888888889
32000: acc=0.64659375
35200: acc=0.6601420454545455
38400: acc=0.673359375
41600: acc=0.6843990384615385
44800: acc=0.6954017857142857
48000: acc=0.70525
51200: acc=0.713828125
54400: acc=0.7218382352941176
57600: acc=0.7288368055555555
60800: acc=0.7352467105263157
64000: acc=0.740484375
67200: acc=0.7462797619047619
70400: acc=0.7513210227272727
73600: acc=0.755991847826087
76800: acc=0.7604296875
80000: acc=0.764575
83200: acc=0.7686298076923077
86400: acc=0.7723611111111112
89600: acc=0.7760491071428571
92800: acc=0.7797737068965517
96000: acc=0.7832916666666667
99200: acc=0.7864415322580646
102400: acc=0.7893359375
105600: acc=0.791969696969697
108800: acc=0.7948161764705882
112000: acc=0.7970267857142858
115200: acc=0.79921875
118400: acc=0.801393581081081


(0.033579795328776044, 0.802575)

## 長短期記憶（LSTM）

經典 RNN 的主要問題之一是所謂的 **消失梯度** 問題。由於 RNN 是在一次反向傳播中端到端訓練的，因此很難將錯誤傳播到網絡的第一層，因此網絡無法學習遠距離標記之間的關係。避免這個問題的方法之一是通過使用所謂的**門**來引入**顯式狀態管理**。有兩種最著名的此類架構：**長短期記憶** (LSTM) 和**門控中繼單元** (GRU)。

![顯示長短期記憶單元示例的圖像](./img/long-short-term-memory-cell.svg)

LSTM 網絡的組織方式類似於 RNN，但有兩種狀態在層與層之間傳遞：實際狀態 $c$ 和隱藏向量 $h$。在每個單元，隱藏向量 $h_i$ 與輸入 $x_i$ 連接，它們通過 **gates** 控制狀態 $c$ 發生的事情。每個門都是一個帶有 sigmoid 激活的神經網絡（輸出在 $[0,1]$ 範圍內），當乘以狀態向量時，可以將其視為按位掩碼。有以下幾個門（上圖從左到右）：
* **forget gate** 獲取隱藏向量並確定我們需要忘記向量 $c$ 的哪些部分，以及要通過哪些部分。
* **輸入門**從輸入和隱藏向量中獲取一些信息，並將其插入到狀態中。
* **輸出門**通過一些帶有$\tanh$激活的線性層來轉換狀態，然後使用隱藏向量$h_i$選擇它的一些組件來產生新的狀態$c_{i+1}$。

狀態 $c$ 的組件可以被認為是一些可以打開和關閉的標誌。例如，當我們在序列中遇到一個名字 *Alice* 時，我們可能想假設它指的是女性角色，並在句子中有女性名詞的狀態下升旗。當我們進一步遇到短語*和Tom*時，我們會舉起我們有復數名詞的標誌。因此，通過操縱狀態，我們應該可以跟踪句子部分的語法特性。

> **注意**：Christopher Olah 撰寫的這篇很棒的文章 [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) 是了解 LSTM 內部結構的重要資源。

雖然 LSTM 單元的內部結構可能看起來很複雜，但 PyTorch 將這個實現隱藏在 LSTMCell 類中，並提供了 LSTM 對象來表示整個 LSTM 層。因此，LSTM 分類器的實現將與我們上面看到的簡單 RNN 非常相似：

In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

現在讓我們訓練我們的網絡。 請注意，訓練 LSTM 也很慢，而且在訓練開始時，您的準確率可能沒有太大提高。 此外，您可能需要使用 `lr` 學習率參數來找到導致合理訓練速度的學習率，但不會導致

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.25125
6400: acc=0.260625
9600: acc=0.27458333333333335
12800: acc=0.294765625
16000: acc=0.3214375
19200: acc=0.3165104166666667
22400: acc=0.3209375
25600: acc=0.3353515625
28800: acc=0.34958333333333336
32000: acc=0.36265625
35200: acc=0.37491477272727275
38400: acc=0.385390625
41600: acc=0.3973317307692308
44800: acc=0.4096205357142857
48000: acc=0.42120833333333335
51200: acc=0.43326171875
54400: acc=0.4460661764705882
57600: acc=0.4578993055555556
60800: acc=0.4713322368421053
64000: acc=0.483796875
67200: acc=0.49681547619047617
70400: acc=0.5101136363636364
73600: acc=0.5225815217391304
76800: acc=0.534375
80000: acc=0.5463875
83200: acc=0.5578605769230769
86400: acc=0.5680324074074075
89600: acc=0.5779352678571429
92800: acc=0.5876185344827586
96000: acc=0.5966145833333333
99200: acc=0.6048487903225807
102400: acc=0.6130859375
105600: acc=0.6208901515151515
108800: acc=0.6282444852941177
112000: acc=0.6352232142857143
115200: acc=0.6421006944444444
118400: acc=0.648

(0.048376932779947916, 0.6515083333333334)

## 打包序列

在我們的示例中，我們必須用零向量填充小批量中的所有序列。雖然這會導致一些內存浪費，但對於 RNN，更重要的是為填充的輸入項創建額外的 RNN 單元，這些單元參與訓練，但不攜帶任何重要的輸入信息。僅將 RNN 訓練到實際序列大小會好得多。

為此，PyTorch 中引入了一種特殊的填充序列存儲格式。假設我們有輸入的填充小批量，它看起來像這樣：
``
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
``
這裡 0 表示填充值，輸入序列的實際長度向量為`[5,3,1]`。

為了用填充序列有效地訓練 RNN，我們想開始用大 minibatch (`[1,6,9]`) 訓練第一組 RNN 單元，然後結束第三個序列的處理，並繼續用短小批量 ( `[2,7]`、`[3,8]`）等。因此，打包序列被表示為一個向量——在我們的例子中是`[1,6,9,2,7,3,8,4,5]`，和長度向量（`[5,3,1]`），從中我們可以輕鬆地重建原始的填充小批量。

要生成打包序列，我們可以使用 `torch.nn.utils.rnn.pack_padded_sequence` 函數。所有循環層，包括 RNN、LSTM 和 GRU，都支持打包序列作為輸入，並生成打包輸出，可以使用 `torch.nn.utils.rnn.pad_packed_sequence` 進行解碼。

為了能夠生成打包序列，我們需要將長度向量傳遞給網絡，因此我們需要一個不同的函數來準備小批量：

In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

實際網絡與上面的“LSTMClassifier”非常相似，但“forward”傳遞將同時接收填充的小批量和序列長度向量。 計算嵌入後，我們計算打包序列，將其傳遞給 LSTM 層，然後將結果解包回來。

> **注意**：我們實際上不使用解包結果`x`，因為我們在下面的計算中使用了隱藏層的輸出。 因此，我們可以從這段代碼中完全刪除解包。 我們把它放在這裡的原因是為了讓您能夠輕鬆修改此代碼，以防您需要在進一步計算中使用網絡輸出。

In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

現在讓我們進行訓練： 

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.3140625
6400: acc=0.39453125
9600: acc=0.45375
12800: acc=0.500546875
16000: acc=0.5460625
19200: acc=0.581875
22400: acc=0.6104017857142857
25600: acc=0.635859375
28800: acc=0.6572569444444445
32000: acc=0.6729375
35200: acc=0.6879261363636363
38400: acc=0.7003385416666666
41600: acc=0.7112980769230769
44800: acc=0.7217410714285715
48000: acc=0.7312083333333333
51200: acc=0.73982421875
54400: acc=0.7470220588235295
57600: acc=0.7538368055555555
60800: acc=0.7603947368421052
64000: acc=0.76609375
67200: acc=0.7717708333333333
70400: acc=0.7764630681818182
73600: acc=0.7807336956521739
76800: acc=0.7851041666666667
80000: acc=0.78895
83200: acc=0.7928365384615385
86400: acc=0.7965393518518519
89600: acc=0.7996651785714286
92800: acc=0.8026724137931035
96000: acc=0.8056666666666666
99200: acc=0.8082862903225806
102400: acc=0.81095703125
105600: acc=0.8132575757575757
108800: acc=0.8156617647058824
112000: acc=0.8177232142857143
115200: acc=0.8197829861111111
118400: acc=0.821

(0.028930082194010418, 0.8229333333333333)

> **注意：** 您可能已經註意到我們傳遞給訓練函數的參數 `use_pack_sequence`。 目前，`pack_padded_sequence` 函數要求長度序列張量位於 CPU 設備上，因此訓練函數在訓練時需要避免將長度序列數據移動到 GPU。 您可以在 [`torchnlp.py`](torchnlp.py) 文件中查看 `train_emb` 函數的實現。

## 雙向和多層 RNN

在我們的示例中，所有循環網絡都在一個方向上運行，從序列的開始到結束。它看起來很自然，因為它類似於我們閱讀和聆聽演講的方式。然而，由於在許多實際情況下我們可以隨機訪問輸入序列，因此在兩個方向上運行循環計算可能是有意義的。這種網絡被稱為**雙向** RNN，它們可以通過將`bidirectional=True`參數傳遞給RNN/LSTM/GRU構造函數來創建。

在處理雙向網絡時，我們需要兩個隱藏狀態向量，每個方向一個。 PyTorch 將這些向量編碼為一個兩倍大的向量，這非常方便，因為您通常會將生成的隱藏狀態傳遞給完全連接的線性層，並且您只需要在創建層時考慮這種大小的增加.

循環網絡，單向或雙向，捕獲序列中的某些模式，並且可以將它們存儲到狀態向量中或傳遞到輸出中。與卷積網絡一樣，我們可以在第一個循環層之上構建另一個循環層來捕獲更高級別的模式，從第一層提取的低級模式構建。這引出了**多層 RNN** 的概念，它由兩個或多個循環網絡組成，其中前一層的輸出作為輸入傳遞到下一層。

![顯示多層長短期記憶 RNN 的圖像](./img/multi-layer-lstm.jpeg)

*Fernando López 來自 [這篇精彩文章](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) 的圖片*

PyTorch 使構建此類網絡成為一項簡單的任務，因為您只需要將 `num_layers` 參數傳遞給 RNN/LSTM/GRU 構造函數即可自動構建多層循環。這也意味著隱藏/狀態向量的大小會按比例增加，並且在處理循環層的輸出時需要考慮到這一點。

## 用於其他任務的 RNN

在本單元中，我們已經看到 RNN 可用於序列分類，但實際上，它們可以處理更多任務，例如文本生成、機器翻譯等。 我們將在下一個單元中考慮這些任務。